In [1]:
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
%cd .
# %load_ext autoreload
# %autoreload 2

/home/jh/code/til/til-23-cv


In [2]:
%%sh
./setup.sh

Not on competition platform, exiting...


## Training Suspect Recognition

In [11]:
import timm
import torch
from til_23_cv.reid import cli_main

In [4]:
# https://lightning.ai/docs/pytorch/stable/cli/lightning_cli_advanced_3.html#run-from-python
cli = cli_main(["-c=cfg/reid.yaml"])
trainer = cli.trainer
model = cli.model
data = cli.datamodule

/home/jh/.conda/envs/til/lib/python3.9/site-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"74891908-2ad1-4dea-9897-2ac2e65327cc"', '--shell=9002', '--transport="tcp"', '--iopub=9004', '--f=/home/jh/.local/share/jupyter/runtime/kernel-v2-3897222ehmAkn1PJ6w.json'], args=['-c=cfg/reid.yaml'].
  rank_zero_warn(
Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### `timm` models

In [5]:
# NOTE:
# {model}_{size}_{patch size}_{im size}.{train method}_{dataset}
# m38m is Merged-38M, combines A LOT of datasets.
# ft refers to fine-tuning on a smaller dataset later.
# so m38m_ft_in22k_in1k means pretrained on Merged-38M, then finetuned on
# ImageNet-22k followed by ImageNet-1k.
# Clip models might be useful for their zero-shot capabilities.
display(timm.list_models(pretrained=True, filter="*dino*"))
# backbone = "eva02_large_patch14_448.mim_m38m_ft_in22k_in1k"
# backbone = "eva02_tiny_patch14_336.mim_in22k_ft_in1k"
# https://huggingface.co/timm/vit_small_patch14_dinov2.lvd142m
# backbone = "vit_small_patch14_dinov2.lvd142m"

['resmlp_12_224.fb_dino',
 'resmlp_24_224.fb_dino',
 'vit_base_patch8_224.dino',
 'vit_base_patch14_dinov2.lvd142m',
 'vit_base_patch16_224.dino',
 'vit_giant_patch14_dinov2.lvd142m',
 'vit_large_patch14_dinov2.lvd142m',
 'vit_small_patch8_224.dino',
 'vit_small_patch14_dinov2.lvd142m',
 'vit_small_patch16_224.dino']

### Preview Augmentations

See `notebooks/data.ipynb` for how to convert `til23plush` dataset to `til23reid`.

In [6]:
print(data.nclasses)

200


In [7]:
import cv2
import numpy as np
while False:
    im = data.preview_transform(1)[0]
    im = im.resize((1024, 1024))
    im = np.array(im)[...,::-1]
    cv2.imshow("example", im)
    key = chr(cv2.waitKey(0))
    if key == "q":
        break
cv2.destroyAllWindows()

### Training

In [ ]:
trainer.fit(model, datamodule=data)

**NOTE: Above saves checkpoints to `runs/lightning_logs/version_N/checkpoints`, pick the best for export!**

### Model Export

In [9]:
ckpt_path = "runs/lightning_logs/version_4/checkpoints/epoch=11-val_sil_score=0.434.ckpt"
save_path = "models/reid.pt"

In [12]:
# Load checkpoint.
ckpt = torch.load(ckpt_path)
model.load_state_dict(ckpt["state_dict"])

<All keys matched successfully>

In [16]:
# Trace & save model.
encoder = model.model
sz = model.hparams.im_size
traced = torch.jit.trace(encoder, torch.rand(1, 3, sz, sz))
torch.jit.save(traced, save_path)

/home/jh/.conda/envs/til/lib/python3.9/site-packages/torch/__init__.py:1209: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message


In [18]:
# Check equality.
traced = torch.jit.load(save_path)
x = torch.rand(1, 3, sz, sz)
print(torch.isclose(traced(x), model.model(x)).all())

tensor(True)
